# <center> Quick Insights from the CIA Factbook <center>
## <center>  Eric Ríos Soderman <center>

## <center> Introduction <center>

#### This project's goal is to glean a few insights from the [CIA](https://www.cia.gov/the-world-factbook/) [factbook](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db) through the use of SQL. As such, it is mostly an intent to test my beginner SQL skills. An area of interest to develop is subqueries. The insights to find relate to population, population growth, population to  area ratios, water to land ratios, and averages for population and area of various countries and territories.

## <center> Summary of Results <center>

#### These are the results obtained from the exercise. Thank you very much for reading. 

| Category                                          | Country(ies)                                          | Number(s)       |
|---------------------------------------------------|-------------------------------------------------------|-----------------|
| Lowest Population                                 | Antartica                                             | 0               |
| Highest Population                                | China                                                 | 1.37 billion    |
| Lowest Population Growth                          | Greenland,  Holy See, Cocos Islands, Pitcairn Islands | 0.0             |
| Highest Population Growth                         | South Sudan                                           | 4.02            |
| Above Average Populations and Below Average Areas | Bangladesh, South Korea, Spain                        | See results.    |
| Highest water to land ratios                      | British Indian  Ocean Territory                       | 90,566 units    |
| Best Birth and Death Rate  Delta                  | Malawi                                                | 33.15           |
| Worst Birth and Death Rate Delta                  | Bulgaria                                              | -5.52           |
| Highest Population  per Area Ratio                | Macau                                                 | 21,968.96 units |

In [30]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [31]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


### Exploring the Data

In [32]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The columns here detail the countries' areas, population, death rate, birth rate and migration rate numbers. 

### Extracting Basic Summary Statistics

In [33]:
%%sql
SELECT MIN(population) AS min_population, 
       MAX(population) AS max_population,
       MIN(population_growth) AS min_population_growth,
       MAX(population_growth) AS max_population_growth
  FROM facts;
    

Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


A minimum population of zero and a maximum of 7 billion are strange numbers. Something may be skewing the results.

In [34]:
%%sql
SELECT id, code, name, population, population_growth
  FROM facts
 WHERE population == (SELECT MIN(population) 
                        FROM facts);


Done.


id,code,name,population,population_growth
250,ay,Antarctica,0,None


This number makes complete sense because the area is [uninhabitable](https://en.wikipedia.org/wiki/Antarctica).

In [35]:
%%sql
SELECT id, code, name, population, population_growth
  FROM facts
 WHERE population == (SELECT MAX(population) 
                        FROM facts);

Done.


id,code,name,population,population_growth
261,xx,World,7256490011,1.08


This was the number definitely skewing the results. We can work around this.

In [36]:
%%sql
SELECT id, code, name, population
  FROM facts
 WHERE population == (SELECT MAX(population) 
                        FROM facts
                       WHERE name != 'World'
                     );

Done.


id,code,name,population
37,ch,China,1367485388


China is the country with the most population.

In [37]:
%%sql
SELECT MIN(population) AS min_population, 
       MAX(population) AS max_population,
       MIN(population_growth) AS min_population_growth,
       MAX(population_growth) AS max_population_growth,
       AVG(population) AS avg_population,
       AVG(Area) AS avg_area
  FROM facts 
 WHERE name != 'World';

Done.


min_population,max_population,min_population_growth,max_population_growth,avg_population,avg_area
0,1367485388,0.0,4.02,32242666.56846473,555093.546184739


These are the real summary statistics. 

### Countries with Above Average Populations and Below Average Areas

In [38]:
%%sql
SELECT name, population, area
  FROM facts
 WHERE population > (SELECT AVG(population) 
                       FROM facts 
                      WHERE name != 'World'
                    )
   AND area < (SELECT AVG(area) 
                 FROM facts 
                WHERE name != 'World'
               );

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


Countries such as the United Kingdom, China and South Korea are reminiscent of how expensive living quarters are in densely populated areas.

### Comparing the Highest Populated Country to the Fastest Growing Country 

In [39]:
%%sql
SELECT name , population, population_growth, area, birth_rate, death_rate
  FROM facts 
 WHERE population == (SELECT MAX(population) 
                       FROM facts 
                      WHERE name != 'World'
                      )
    OR population_growth == (SELECT MAX(population_growth) 
                             FROM facts 
                             WHERE name != 'World' 
                             );
   

Done.


name,population,population_growth,area,birth_rate,death_rate
China,1367485388,0.45,9596960,12.49,7.53
South Sudan,12042910,4.02,644329,36.91,8.18


Although South Sudan's area is smaller, it's birth rate is substantially higher than China's number.

### Countries with the Highest Water to Land Ratios

In [40]:
%%sql
SELECT name, ROUND(CAST(area_water AS FLOAT) / area_land * 100,3) 
       AS water_to_land_ratio      
  FROM facts 
 WHERE name != 'World'
 ORDER BY water_to_land_ratio DESC
 LIMIT 10;
 

Done.


name,water_to_land_ratio
British Indian Ocean Territory,90566.667
Virgin Islands,452.023
Puerto Rico,55.479
"Bahamas, The",38.661
Guinea-Bissau,28.467
Malawi,25.94
Netherlands,22.571
Uganda,22.292
Eritrea,16.436
Liberia,15.624


The most relevant detail is the high number of island territories observed here, particularly the British Indian Ocean Territory. 

In [41]:
Countries with the Lowest and Highest Population Growths

SyntaxError: invalid syntax (<ipython-input-41-bce173d70f77>, line 1)

In [ ]:
%%sql
SELECT name, population, population_growth, birth_rate, death_rate     
  FROM facts 
 WHERE population_growth == 0 #To avoid None results
 ORDER BY population_growth
 LIMIT 10;

There are many islands and territories for which the data is either zero or nonexistent. 

In [ ]:
%%sql
SELECT name, population, population_growth, birth_rate, death_rate   
  FROM facts 
 ORDER BY population_growth DESC
 LIMIT 10;

South Sudan, Malawi, and Burundi are the standouts. In addition, African countries dominate this category of highest population growth.

### Countries with the Best and Worst Birth and Death Rate Deltas

In [ ]:
%%sql
SELECT name, population, population_growth, 
      (birth_rate - death_rate) AS birth_death_rate_delta     
  FROM facts 
 ORDER BY birth_death_rate_delta DESC
 LIMIT 10;

Malawi, Uganda, and Niger have the best deltas, and African countries have a strong presence in this category.

In [ ]:
%%sql
SELECT name, population, population_growth, 
      (birth_rate - death_rate) AS birth_death_rate_delta     
  FROM facts
 WHERE birth_death_rate_delta < 1
 ORDER BY birth_death_rate_delta 
 LIMIT 10;

Bulgaria has the worst numbers, but Eastern Europe as a whole is not faring well with its highest numbers peaking at -2.73.

### Countries with the Highest Population to Area Ratios

In [ ]:
%%sql
SELECT name, ROUND(CAST(population AS FLOAT)/area,2) 
       AS population_area_ratio 
  FROM facts 
 ORDER BY population_area_ratio DESC
 LIMIT 10;

Many areas here hail from the Eastern Hemisphere.

## <center> Conclusion <center>

#### After conducting the exercise, many interesting insights were gleaned. It was a very enjoyable process to build proficiency in a language that carries a high degree of importance in the technology sector. Here I will provide a table that summarizes the results. Lastly, I thank the reader for their attention. 

| Category                                          | Country(ies)                                          | Number(s)       |
|---------------------------------------------------|-------------------------------------------------------|-----------------|
| Lowest Population                                 | Antartica                                             | 0               |
| Highest Population                                | China                                                 | 1.37 billion    |
| Lowest Population Growth                          | Greenland,  Holy See, Cocos Islands, Pitcairn Islands | 0.0             |
| Highest Population Growth                         | South Sudan                                           | 4.02            |
| Above Average Populations and Below Average Areas | Bangladesh, South Korea, Spain                        | See results.    |
| Highest water to land ratios                      | British Indian  Ocean Territory                       | 90,566 units    |
| Best Birth and Death Rate  Delta                  | Malawi                                                | 33.15           |
| Worst Birth and Death Rate Delta                  | Bulgaria                                              | -5.52           |
| Highest Population  per Area Ratio                | Macau                                                 | 21,968.96 units |